In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



In [9]:
cat > motif_marginal_lm.R << 'EOF'

###################################################
# Set environment
###################################################
cat("\n++++++++++ Set environment  ++++++++++\n")

#source("/home/mount/project/config_sing.R")
source("config_sing.R")

###################################################
# Set global variables
###################################################
cat("\n++++++++++ Set global variables ++++++++++\n")

### Get argument: Chromomsome
ARGS            = commandArgs(trailingOnly=TRUE)
TARGET          = as.character(ARGS[1])  # which chromosome or region to run
FDIRY           = as.character(ARGS[2])  # the name of the output folder
N_CORE          = as.integer(ARGS[3])    # number of cores to register during the parallelization
THRESHOLD_COVER = as.numeric(ARGS[4])    # threshold for the motif score filteration

### set global variables
SAMPLES_INP = c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

SAMPLES_INP20X = c(
    paste0("Input", 1:5, "_20x"),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

LST_SAMPLES = list(input=SAMPLES_INP, input20x=SAMPLES_INP20X)

FD_OUT = file.path(FD_RES, "model_linear", FDIRY, TARGET)
dir.create(FD_OUT, recursive = TRUE, showWarnings = FALSE)

### set motifs
fdiry  = file.path(FD_RES, "model_linear", FDIRY, TARGET)
fname  = "count_*"
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
MOTIFS = basename(fpaths)

### set column names and types
CTYPES = c(col_character(), col_character(), col_integer(), col_integer())
CNAMES = c("Sample", "Motif", "N_Motif", "Value")
           
### print start message
cat("Target:           ", TARGET,          "\n")
cat("Output Directory: ", FD_OUT,          "\n")
cat("#Cores Resgister: ", N_CORE,          "\n")
cat("Threshold (Cover):", THRESHOLD_COVER, "\n")

###################################################
# Import library size
###################################################
cat("\n++++++++++ Import library size ++++++++++\n")

### Helper function to get
get_group = function(idn_sample){
    idn = idn_sample
    
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    
    idn = str_remove(
        string = idn, 
        pattern = "_20x")
    
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "FPath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names=cnames)

### remove the total size
dat_lib = dat_lib %>% dplyr::filter(FPath != "total")

### summarize info from the file path
### stackoverflow: Extract only folder name right before filename from full path
dat_lib = dat_lib %>% 
    mutate(Sample = basename(dirname(FPath))) %>%
    mutate(Group = get_group(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)

###################################################
# Get motif count table
###################################################
cat("\n++++++++++ Get motif count table ++++++++++\n")

### start
registerDoParallel(N_CORE)
timer_start = Sys.time()

### loop through each motif to get the marginal effect
#lst_tmp = foreach(motif = MOTIFS) %do% {

#for (motif in MOTIFS) {
lst_tmp = foreach (motif = MOTIFS) %dopar% {
    ### start message and get the name of motif
    ### example: motif = "AHR_merge.bed.gz"
    mtf = str_remove_all(motif, pattern = "count_|\\.tsv")
    msg = paste(mtf, "Start")
    cat(msg, "\n"); flush.console()
    
    #++++++++++++++++++++++++++++++++++++++++++
    
    ### set directory
    fdiry  = file.path(FD_RES, "model_linear", FDIRY, TARGET)
    fname  = motif
    fpath  = file.path(fdiry, fname)
    
    ### PRINT
    msg = paste(mtf, "Import", fpath)
    cat(msg, "\n"); flush.console()
    
    ### import
    dat = read_tsv(fpath, show_col_types = FALSE)
    
    #++++++++++++++++++++++++++++++++++++++++++
    
    ### PRINT
    msg = paste(mtf, "Set")
    cat(msg, "\n"); flush.console()
    
    ### get total count fragments for each sample
    dat = dat %>% group_by(Sample) %>% summarize(Value = sum(Value))
    
    ### normalize counts by library size
    dat = dat %>% left_join(dat_lib, by="Sample")
    dat = dat %>%
        mutate(Norm_Value    = Value / Size) %>% 
        mutate(Lognorm_Value = log2(Value) - log2(Size)) %>%
        mutate(X = Group)
    
    #++++++++++++++++++++++++++++++++++++++++++
    
    lst_res = lapply(LST_SAMPLES, function(SAMPLES){
        ###
        tmp = dat %>% dplyr::filter(Sample %in% SAMPLES)

        ### create design matrix
        idxs  = c("Input", "TFX_DMSO", "TFX_Dex")
        tmp$X = factor(tmp$X, levels=idxs)
        X = model.matrix(~X, tmp)

        ### setup design matrix
        X[,"XTFX_DMSO"] = X[,"XTFX_DMSO"] + X[,"XTFX_Dex"]

        ### fit model and get the summary
        y = tmp$Norm_Value    
        fit = lm(y ~ X + 0)

        y = tmp$Lognorm_Value
        fit_log = lm(y ~ X + 0)

        ### arrange
        lst = list()
        lst$cnt     = tmp
        lst$fit     = fit
        lst$fit_log = fit_log
        lst$X       = X
        return(lst)
    })
    
    #++++++++++++++++++++++++++++++++++++++++++
    
    ### store the results
    fdiry = FD_OUT
    fname = paste0("lm_", mtf, ".RDS") # str_replace(mtf, pattern = "/", replacement = "_")
    fpath = file.path(fdiry, fname)
    
    ### PRINT
    msg = paste(mtf, "Store")
    cat(msg, "\n"); flush.console()
    
    ###
    saveRDS(lst_res, fpath)
    
} # end loop (foreach)

### PRINT
msg = "Done!"
cat(msg, "\n"); flush.console()

### print end message
timer = Sys.time()
cat("Timer of the loop:\n")
print(timer - timer_start)

EOF

In [12]:
echo hello

hello


In [33]:
### set log file directory
sbatch -p ${NODE} \
    --mem=8G \
    --tasks-per-node=1 \
    --cpus-per-task=8 \
    --job-name='Marginal lm PER1 f00' \
    -o ${FD_LOG}/linear_model_marginal_lm_per1_filter00.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v

#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#CHROM=chrY

TARGET=target_PER1
#IS_INPUT20X=TRUE
FDIRY=marginal_filter00
NCORE=5
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26624742


In [4]:
### set log file directory
sbatch -p ${NODE} \
    --mem=8G \
    --tasks-per-node=1 \
    --cpus-per-task=8 \
    --job-name='Marginal lm PER1 f00' \
    -o ${FD_LOG}/linear_model_marginal_lm_per1_filter00.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v

#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#CHROM=chrY

TARGET=target_PER1
FDIRY=example
NCORE=5
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26651262


## Test chromsome 22

In [11]:
### set log file directory
sbatch -p ${NODE} \
    --mem=8G \
    --tasks-per-node=1 \
    --cpus-per-task=8 \
    --job-name='Marginal lm chr22 f95' \
    -o ${FD_LOG}/linear_model_marginal_lm_chr22_filter95.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v

#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#CHROM=chrY

TARGET=chr22
FDIRY=example
NCORE=5
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26651269


In [7]:
### set log file directory
sbatch -p ${NODE} \
    --mem=8G \
    --tasks-per-node=1 \
    --cpus-per-task=8 \
    --job-name='Marginal lm PER1 f95' \
    -o ${FD_LOG}/linear_model_marginal_lm_input20x_per1_filter95.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v

#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#CHROM=chrY

TARGET=target_PER1
IS_INPUT20X=TRUE
FDIRY=marginal_filter95
NCORE=5
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${IS_INPUT20X} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26611815


In [4]:
### set log file directory
sbatch -p ${NODE} \
    --mem=20G \
    --tasks-per-node=1 \
    --cpus-per-task=21 \
    --job-name='Marginal lm chrY f95' \
    -o ${FD_LOG}/linear_model_marginal_lm_input20x_chrY_filter95.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v

#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#CHROM=chrY

TARGET=chrY
IS_INPUT20X=TRUE
FDIRY=marginal_filter95
NCORE=20
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${IS_INPUT20X} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26611426


In [8]:
### set log file directory
sbatch -p ${NODE} \
    --mem=20G \
    --tasks-per-node=1 \
    --cpus-per-task=8 \
    --job-name='Marginal lm chr22 f95' \
    -o ${FD_LOG}/linear_model_marginal_lm_input20x_chr22_filter95.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v

#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#CHROM=chrY

TARGET=chr22
IS_INPUT20X=TRUE
FDIRY=marginal_filter95
NCORE=5
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${IS_INPUT20X} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26611817


In [5]:
### set log file directory
sbatch -p ${NODE} \
    --mem=20G \
    --tasks-per-node=1 \
    --cpus-per-task=21 \
    --job-name='Marginal lm chrY f95' \
    -o ${FD_LOG}/linear_model_marginal_lm_input20x_chrY_filter95.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v

#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#CHROM=chrY

TARGET=chr17
IS_INPUT20X=TRUE
FDIRY=marginal_filter95
NCORE=20
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${IS_INPUT20X} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26611475


## Run all chromosomes

In [13]:
### set log file directory
sbatch -p ${NODE} \
    --array=0-23 \
    --mem=10G \
    --tasks-per-node=1 \
    --cpus-per-task=16 \
    --job-name='Marginal lm chrom f95' \
    -o ${FD_LOG}/linear_model_marginal_lm_chrom_filter95.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}

TARGET=${CHROM}
FDIRY=marginal_filter95
NCORE=10
THRESHOLD_COVER=0

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo "Chromosome:        " ${CHROM}
echo

### run the model
srun ${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal_lm.R \
    ${TARGET} ${FDIRY} ${NCORE} ${THRESHOLD_COVER}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26656089


In [14]:
tail ${FD_LOG}/linear_model_marginal_lm_chrom_filter95.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/linear_model_marginal_lm_chrom_filter95.0.txt <==
ZSCAN4 Start 
ZSCAN4 Import /home/mount/work/out/proj_combeffect/model_linear/marginal_filter95/chr1/count_ZSCAN4.tsv 
ZSCAN4 Set 
ZSCAN4 Store 
Done! 
Timer of the loop:
Time difference of 1.693413 secs

Done!
Run Time: 7 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/linear_model_marginal_lm_chrom_filter95.10.txt <==
ZSCAN4 Start 
ZSCAN4 Import /home/mount/work/out/proj_combeffect/model_linear/marginal_filter95/chr11/count_ZSCAN4.tsv 
ZSCAN4 Set 
ZSCAN4 Store 
Done! 
Timer of the loop:
Time difference of 1.637216 secs

Done!
Run Time: 7 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/linear_model_marginal_lm_chrom_filter95.11.txt <==
ZNF667 Import /home/mount/work/out/proj_combeffect/model_linear/marginal_filter95/chr12/count_ZNF667.tsv 
ZNF667 Set 
ZNF768 Store 
ZNF667 Store 
Done! 
Timer of the loop:
Time difference of 1.584581 secs

Done!
